Em's Debug File - currently working on concatenating ang and e_p onto latent z

In [20]:
import numpy as np
import tensorflow as tf
import h5py

In [31]:
def GetDataAngle(datafile, imgs3dscale =1, imgs3dpower=1, e_pscale = 100, angscale=1, angtype='theta', thresh=1e-4):
    print ('Loading Data from .....', datafile)
    f = h5py.File(datafile,'r')                    # load data into f variable
    ang = np.array(f.get(angtype))                 # ang is an array of angle data from f, one value is concatenated onto the latent vector
    imgs3d = np.array(f.get('ECAL'))* imgs3dscale    # imgs3d is a 3d array, cut from the cylinder that the calorimeter produces -has 25 layers along z-axis
    e_p = np.array(f.get('energy'))/e_pscale       # e_p is an array of scaled energy data from f, one value is concatenated onto the latent vector
    imgs3d[imgs3d < thresh] = 0        # when imgs3d values are less than the threshold, they are reset to 0
    
    # set imgs3d, e_p, and ang as float 32 datatypes
    imgs3d = imgs3d.astype(np.float32)
    e_p = e_p.astype(np.float32)
    ang = ang.astype(np.float32)
    
    imgs3d = np.expand_dims(imgs3d, axis=-1)         # insert a new axis at the beginning for imgs3d
    
    # sum along axis
    ecal = np.sum(imgs3d, axis=(1, 2, 3))    # summed imgs3d data, used for training the discriminator
     
    # imgs3d ^ imgs3dpower
    if imgs3dpower !=1.:
        imgs3d = np.power(imgs3d, imgs3dpower)
            
    # imgs3d=ecal data; e_p=energy data; ecal=summed imgs3d (used to train the discriminator); ang=angle data
    return imgs3d, e_p, ang, ecal

In [32]:
imgs3d, e_p, ang, ecal = GetDataAngle('Ele_VarAngleMeas_100_200_005.h5')

Loading Data from ..... Ele_VarAngleMeas_100_200_005.h5


In [23]:
latent_dim = 256
z = tf.random.normal([latent_dim-2])
z = tf.concat([z, [e_p[0]], [ang[0]]], 0)
z

<tf.Tensor 'concat_13:0' shape=(256,) dtype=float32>

In [24]:
#z = tf.random.normal(shape=[tf.shape(real_image_input)[0], latent_dim-2])    #shape=[int corresponding to batch_size, 254]
z_batch_size = 4                                             #tf.shape(real_image_input)[0]
z = tf.random.normal(shape=[z_batch_size, latent_dim-2])   
print(z.shape)
e_p_vector = e_p[0:(z_batch_size)].reshape(z_batch_size,1)   # need z_batch_size x 1
print(e_p_vector.shape)
ang_vector = ang[0:(z_batch_size)].reshape(z_batch_size,1)   # need z_batch_size x 1
print(ang_vector.shape)
z = tf.concat([ z, e_p_vector, ang_vector ], 1)
print(z.shape)
print(z)

(4, 254)
(4, 1)
(4, 1)
(4, 256)
Tensor("concat_14:0", shape=(4, 256), dtype=float32)


Loss.py concatenation code -- subbed phase and batch_size variables

In [25]:
phase = 2 #added
z_batch_size = 12#tf.shape(real_image_input)[0]                 # this value should be an integer
z = tf.random.normal(shape=[z_batch_size, latent_dim-2])   
start = (z_batch_size * (phase-1))
e_p_vector = e_p[start:(start+z_batch_size)].reshape(z_batch_size,1)   # need z_batch_size x 1
ang_vector = ang[start:(start+z_batch_size)].reshape(z_batch_size,1)   # need z_batch_size x 1
z = tf.concat([ z, e_p_vector, ang_vector ], 1)    # shape = (z_batch_size, 256)
z

<tf.Tensor 'concat_15:0' shape=(12, 256) dtype=float32>

Emma's new code -- take out batch selection (adel just passes the correct batch size in sara_main.py)

In [43]:
phase = 2 #added for testing
# the batch size will be much smaller in reality, I only put in 5000 for testing because Adel sends in the shortened tensors
z_batch_size = 5000#tf.shape(real_image_input)[0]                 # this value should be an integer

#Adel is passing e_p and ang as batch sized tensors
e_p_tensor = tf.reshape(e_p, [z_batch_size,1])   # need z_batch_size x 1
print(e_p_tensor.shape)
print(type(e_p_tensor))
ang_tensor = tf.reshape(ang, [z_batch_size,1])   # need z_batch_size x 1
print(ang_tensor.shape)

z = tf.random.normal(shape=[z_batch_size, latent_dim-2])   
z = tf.concat([z, e_p_tensor, ang_tensor], 1)    # shape = (z_batch_size, 256)
print(z.shape)

(5000, 1)
<class 'tensorflow.python.framework.ops.Tensor'>
(5000, 1)
(5000, 256)
